In [37]:
import pandas as pd
from functools import reduce
import sys
sys.path.append("../..")
from config.settings import get_data_url, Club, attr, MatchLogTypes
pd.set_option('display.max_columns', 70)
df = pd.read_csv("../../data/Combined_Team_Stats.csv")
df_matches = pd.read_csv("../../data/Combined_MLS_Data2024.csv")
df_odds = pd.read_excel("../../data/mls_historical_odds.xlsx")



df_odds.head(5)

,Season,Date,Home Team,Away Team,Full Time Home Team Goals Scored,Full Time Away Team Goals Scored,Full Time Result,Industry Maximum Home Team Win Odds,Industry Maximum Draw Odds,Industry Maximum Away Team Win Odds,Industry Average Home Team Win Odds,Industry Average Draw Odds,Industry Average Away Team Win Odds,Industry Maximum Home Team Win Odds (American),Industry Maximum Draw Odds (American),Industry Maximum Away Team Win Odds (American),Industry Average Home Team Win Odds (American),Industry Average Draw Odds (American),Industry Average Away Team Win Odds (American)
0,2023,2/25/2023,Nashville SC,New York City,2,0,Home Team Win,2.20,3.50,4.04,2.08,3.29,3.67,120,250,304,108,229,267
1,2023,2/26/2023,Atlanta Utd,San Jose Earthquakes,2,1,Home Team Win,1.72,4.35,5.25,1.65,4.13,4.75,-139,335,425,-154,313,375
2,2023,2/26/2023,Charlotte,New England Revolution,0,1,Away Team Win,2.12,3.78,3.83,2.00,3.61,3.60,112,278,283,-100,261,260
3,2023,2/26/2023,DC United,Toronto FC,3,2,Home Team Win,2.85,3.85,2.63,2.64,3.63,2.48,185,285,163,164,263,148
4,2023,2/26/2023,FC Cincinnati,Houston Dynamo,2,1,Home Team Win,1.58,4.70,6.00,1.54,4.31,5.67,-172,370,500,-185,331,467


In [38]:
def parse_date_with_multiple_formats(date_str):
    for fmt in ('%m/%d/%Y', '%Y-%m-%d %H:%M:%S'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError as e:
            print(f"{date_str} , Error occurred: {e}")
            continue
    return pd.NaT  # Return Not a Time for unparseable formats


In [39]:
standard_team_names = {
    # Variants for Montreal
    'montreal impact': 'CF Montreal',
    'club de foot montreal': 'CF Montreal',
    'cf montréal': 'CF Montreal',
    'cf montrãal': 'CF Montreal',
    'montreal impact': 'CF Montreal',
    'CF MontrÃ©al': 'CF Montreal',
    'cf montrã©al': 'CF Montreal',

    # Houston Dynamo
    'dynamo': 'Houston Dynamo',
    'dynamo fc': 'Houston Dynamo',

    # NYCFC
    'new york city': 'NYCFC',

    # NY Red Bulls
    'new york red bulls': 'NY Red Bulls',
    'ny red bulls': 'NY Red Bulls',

    # Atlanta
    'atlanta utd': 'Atlanta United',
    'atlanta united': 'Atlanta United',

    # LA Galaxy / FC
    'los angeles galaxy': 'LA Galaxy',
    'lafc': 'Los Angeles FC',
    'la galaxy': 'LA Galaxy',

    # Seattle
    'seattle': 'Seattle Sounders',

    # Portland
    'portland': 'Portland Timbers',

    # Chicago
    'chicago': 'Chicago Fire',

    # Columbus
    'columbus': 'Columbus Crew',

    # Minnesota
    'minnesota utd': 'Minnesota United',

    # Vancouver
    'vancouver': 'Vancouver Whitecaps',
    "vancouver w'caps": 'Vancouver Whitecaps',

    # San Jose
    'san jose': 'San Jose Earthquakes',
    'sj earthquakes': 'San Jose Earthquakes',

    # Philadelphia
    'philadelphia': 'Philadelphia Union',

    # NE Revolution
    'new england': 'New England Revolution',
    'ne revolution': 'New England Revolution',

    # DC United
    'dc united': 'D.C. United',

    # Sporting KC
    'sporting kc': 'Sporting Kansas City',

    # Nashville
    'nashville': 'Nashville SC',

    # St. Louis
    'st louis': 'St. Louis City',
    'St. Louis': 'St. Louis City',

    # Austin
    'austin': 'Austin FC',

    # General cleanup
    'fc dallas': 'FC Dallas',
    'toronto fc': 'Toronto FC',
    'inter miami': 'Inter Miami',
    'orlando city': 'Orlando City',
    'cf montréal': 'CF Montreal',
    'philadelphia union': 'Philadelphia Union'
}


In [40]:

df_odds["Home Team"] = df_odds["Home Team"].str.lower().replace(standard_team_names)
df_odds["Away Team"] = df_odds["Away Team"].str.lower().replace(standard_team_names)
df["Team"] = df["Team"].str.lower().replace(standard_team_names)
df_matches["Team"] = df_matches["Team"].str.lower().replace(standard_team_names)

df_matches["Team"] = df_matches["Team"].str.lower()
df["Team"] = df["Team"].str.lower()
df_odds["Home Team"] = df_odds["Home Team"].str.lower()
df_odds["Away Team"] = df_odds["Away Team"].str.lower()

df_matches.replace('st. louis','st. louis city', inplace=True)

print(sorted(df_matches["Team"].unique()))



['atlanta united', 'austin fc', 'cf montreal', 'charlotte', 'chicago fire', 'colorado rapids', 'columbus crew', 'd.c. united', 'fc cincinnati', 'fc dallas', 'houston dynamo', 'inter miami', 'la galaxy', 'los angeles fc', 'minnesota united', 'nashville sc', 'new england revolution', 'ny red bulls', 'nycfc', 'orlando city', 'philadelphia union', 'portland timbers', 'real salt lake', 'san jose earthquakes', 'seattle sounders', 'sporting kansas city', 'st. louis city', 'toronto fc', 'vancouver whitecaps']


In [41]:
df_odds.head(20)

,Season,Date,Home Team,Away Team,Full Time Home Team Goals Scored,Full Time Away Team Goals Scored,Full Time Result,Industry Maximum Home Team Win Odds,Industry Maximum Draw Odds,Industry Maximum Away Team Win Odds,Industry Average Home Team Win Odds,Industry Average Draw Odds,Industry Average Away Team Win Odds,Industry Maximum Home Team Win Odds (American),Industry Maximum Draw Odds (American),Industry Maximum Away Team Win Odds (American),Industry Average Home Team Win Odds (American),Industry Average Draw Odds (American),Industry Average Away Team Win Odds (American)
0,2023,2/25/2023,nashville sc,nycfc,2,0,Home Team Win,2.20,3.50,4.04,2.08,3.29,3.67,120,250,304,108,229,267
1,2023,2/26/2023,atlanta united,san jose earthquakes,2,1,Home Team Win,1.72,4.35,5.25,1.65,4.13,4.75,-139,335,425,-154,313,375
2,2023,2/26/2023,charlotte,new england revolution,0,1,Away Team Win,2.12,3.78,3.83,2.00,3.61,3.60,112,278,283,-100,261,260
3,2023,2/26/2023,d.c. united,toronto fc,3,2,Home Team Win,2.85,3.85,2.63,2.64,3.63,2.48,185,285,163,164,263,148
4,2023,2/26/2023,fc cincinnati,houston dynamo,2,1,Home Team Win,1.58,4.70,6.00,1.54,4.31,5.67,-172,370,500,-185,331,467
5,2023,2/26/2023,inter miami,cf montreal,2,0,Home Team Win,2.50,3.63,3.14,2.33,3.44,2.97,150,263,214,133,244,197
6,2023,2/26/2023,orlando city,ny red bulls,1,0,Home Team Win,2.45,3.34,3.25,2.36,3.19,3.12,145,234,225,136,219,212
7,2023,2/26/2023,philadelphia union,columbus crew,4,1,Home Team Win,1.57,4.63,6.54,1.52,4.34,5.97,-175,363,554,-192,334,497
8,2023,2/26/2023,austin fc,st. louis city,2,3,Away Team Win,1.53,4.70,7.39,1.49,4.42,6.34,-189,370,639,-204,342,534
9,2023,2/26/2023,fc dallas,minnesota united,0,1,Away Team Win,1.87,3.77,4.82,1.81,3.54,4.51,-115,277,382,-123,254,351


In [42]:
df_clean = df[["Team","Season","npxG","SCA","GCA","GF","Sh","xA","SoT", "Pts"]]
print(df_clean)
df_clean.to_csv("../../data/MLS_1823_clean.csv")

                    Team  Season  npxG  SCA  GCA  GF   Sh    xA  SoT  Pts
0            cf montreal    2018  39.9  717   80  47  418  28.3  130   46
1         houston dynamo    2018  53.7  825   96  58  470  35.8  178   38
2             toronto fc    2018  55.3  852   99  59  476  41.0  172   36
3              fc dallas    2018  52.8  849   90  52  477  32.4  168   57
4         atlanta united    2018  68.2  902  111  70  497  44.6  187   69
..                   ...     ...   ...  ...  ...  ..  ...   ...  ...  ...
181          cf montreal    2024  41.4  671   88  50  371  32.5  131   44
182            austin fc    2024  35.0  615   73  39  338  28.5  100   42
183       atlanta united    2024  51.8  956   92  54  537  42.4  172   47
184     minnesota united    2024  52.5  808   97  61  483  38.6  179   54
185  vancouver whitecaps    2024  52.5  848   94  61  481  45.8  156   53

[186 rows x 10 columns]


In [43]:

df_matches_clean = df_matches[["Team","Opp","Season","npxG","SCA","GCA","GF","Sh.2","xA","SoT", "Result", "Date"]]
df_matches_clean.to_csv("../../data/MLS_Matches_1823_clean.csv")

In [44]:

filtered_df = df_odds[~df_odds['Season'].between(2012, 2017)]
filtered_df['Date'] = filtered_df['Date'].apply(parse_date_with_multiple_formats)

C:\Users\jamal\AppData\Local\Temp\ipykernel_17732\811614169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = filtered_df['Date'].apply(parse_date_with_multiple_formats)


In [45]:
df_odds_clean = filtered_df[["Season", "Date", "Home Team", "Away Team", "Full Time Result", "Industry Average Away Team Win Odds (American)", "Industry Average Draw Odds (American)" , "Industry Average Home Team Win Odds (American)"]]
df_odds_clean.to_csv("../../data/MLS_Matches_odds_clean.csv")

In [46]:
unique_set_df1 = set(df_odds_clean['Home Team'])
unique_set_df2 = set(df_matches_clean['Team'])

common_elements = unique_set_df1.intersection(unique_set_df2)

print(len(common_elements))

29


In [47]:
def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [
        col[1].strip() if isinstance(col, tuple) and len(col) > 1 else str(col).strip()
        for col in df.columns
    ]
    return df

def normalize_date_column(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if "date" in col.lower():
            df.rename(columns={col: "Date"}, inplace=True)
            df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
            break
    return df

In [48]:
def load_fbref_table(url: str, attrs: dict = {"id": "matchlogs_for"}) -> pd.DataFrame:
    try:
        tables = pd.read_html(url, attrs=attrs)
        df = tables[0]
        df = flatten_columns(df)
        df = normalize_date_column(df)
        return df
    except Exception as e:
        print(f"Error loading table from {url}: {e}")
        return pd.DataFrame()

In [49]:
def merge_fbref_matchlogs(club: Club, year: int, log_types: list[MatchLogTypes]) -> pd.DataFrame:
    dfs = []
    for log_type in log_types:
        url = get_data_url(club, log_type.value, year)
        df = load_fbref_table(url)
        if not df.empty:
            dfs.append(df)
    if not dfs:
        return pd.DataFrame()

    return reduce(lambda left, right: pd.merge(left, right, on="Date", how="outer"), dfs)

In [50]:
merged_df = merge_fbref_matchlogs(
    Club.Philadelphia,
    2024,
    [MatchLogTypes.Shooting, MatchLogTypes.GoalAndShotCreation]
)


In [51]:
print(merged_df)

         Date Time_x Comp_x         Round_x Day_x Venue_x Result_x  GF_x  \
0  2024-02-24  19:30    MLS  Regular Season   Sat    Home        D   2.0   
1  2024-03-02  19:30    MLS  Regular Season   Sat    Away        D   1.0   
2  2024-03-16  19:30    MLS  Regular Season   Sat    Away        D   2.0   
3  2024-03-23  19:30    MLS  Regular Season   Sat    Away        W   3.0   
4  2024-03-30  14:00    MLS  Regular Season   Sat    Home        W   2.0   
5  2024-04-06  19:30    MLS  Regular Season   Sat    Away        W   2.0   
6  2024-04-14  14:30    MLS  Regular Season   Sun    Away        D   2.0   
7  2024-04-27  19:30    MLS  Regular Season   Sat    Home        L   1.0   
8  2024-04-30  19:30    MLS  Regular Season   Tue    Home        L   2.0   
9  2024-05-04  19:30    MLS  Regular Season   Sat    Away        D   2.0   
10 2024-05-11  19:30    MLS  Regular Season   Sat    Home        L   2.0   
11 2024-05-15  19:30    MLS  Regular Season   Wed    Home        L   1.0   
12 2024-05-1